**Алгоритм MinHash**

In [13]:
import hashlib
import random

In [14]:
class MinHash:
    def __init__(self, k=100):
        self.k = k
        self.params = [(random.randint(1, 2**31-1), random.randint(1, 2**31-1)) 
                       for _ in range(k)]
        self.mod = 2**32 - 1
    
    def h(self, x, a, b):
        x_int = int(hashlib.sha256(str(x).encode()).hexdigest(), 16) % self.mod
        return (a * x_int + b) % self.mod
    
    def h_min(self, set):
        min_hashes = [float('inf')] * self.k
        
        for elem in set:
            for i, (a, b) in enumerate(self.params):
                hash_value = self.h(elem, a, b)
                if hash_value < min_hashes[i]:
                    min_hashes[i] = hash_value
        
        return min_hashes
    
    def jaccard(self, min_A, min_B):
        matches = sum(1 for i in range(self.k) if min_A[i] == min_B[i])
        return matches / self.k

In [15]:
def test_identical_sets():
    """Тест 1: Одинаковые множества -> J=1"""
    mh = MinHash(k=100)
    A = {1, 2, 3, 4, 5}
    B = {1, 2, 3, 4, 5}
    
    min_A = mh.h_min(A)
    min_B = mh.h_min(B)
    j = mh.jaccard(min_A, min_B)
    
    assert j == 1.0
    print("Тест 1: Одинаковые множества OK")


def test_disjoint_sets():
    """Тест 2: Непересекающиеся множества -> J≈0"""
    mh = MinHash(k=100)
    A = {1, 2, 3, 4, 5}
    B = {6, 7, 8, 9, 10}
    
    min_A = mh.h_min(A)
    min_B = mh.h_min(B)
    j = mh.jaccard(min_A, min_B)
    
    assert j < 0.15
    print("Тест 2: Непересекающиеся множества OK")


def test_partial_overlap():
    """Тест 3: Частичное пересечение (J=0.25)"""
    mh = MinHash(k=1000)
    A = {1, 2, 3, 4, 5}
    B = {4, 5, 6, 7, 8}
    
    min_A = mh.h_min(A)
    min_B = mh.h_min(B)
    j = mh.jaccard(min_A, min_B)
    
    error = abs(j - 0.25)
    assert error < 0.05
    print(f"Тест 3: Частичное пересечение OK")


def run_all_tests():
    tests = [test_identical_sets, test_disjoint_sets, test_partial_overlap]
    
    for test in tests:
        try:
            test()
        except Exception as e:
            print(f"{test.__name__} провален: {e}")
            return
    print("ВСЕ ТЕСТЫ ПРОЙДЕНЫ")


if __name__ == "__main__":
    run_all_tests()

Тест 1: Одинаковые множества OK
Тест 2: Непересекающиеся множества OK
Тест 3: Частичное пересечение OK
ВСЕ ТЕСТЫ ПРОЙДЕНЫ
